In [226]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as si
import yfinance as yf

In [227]:
S0 = 3110.87              # spot stock price
K = 3100.00                # strike ATM
T = 4/52                 # maturity 
r = 0.0174                 # risk free rate 
sig = 0.3575               # diffusion coefficient or volatility
N = 3                   # number of periods or number of time steps  
payoff = 'put'          # payoff

In [228]:
def option_bs(S, K, T, r, sig, payoff):
    
    d1 = (np.log(S / K) + (r + 0.5 * sig ** 2) * T) / (sig * np.sqrt(T))
    d2 = (np.log(S / K) + (r - 0.5 * sig ** 2) * T) / (sig * np.sqrt(T))
    if payoff == "call":
        option_value = np.exp(-r * T) * si.norm.cdf(d2, 0.0, 1.0)
    elif payoff == "put":
        option_value = np.exp(-r * T) * si.norm.cdf(-d2, 0.0, 1.0)
    
    return option_value

In [229]:
option_bs(3110.87, 3100, 4/52, 0.0174, 0.3575, 'put')

0.4996399908424095

In [230]:
def mcs_simulation_np(p):
    M = p
    I = p
    dt = T / M 
    S = np.zeros((M + 1, I))
    S[0] = S0 
    rn = np.random.standard_normal(S.shape) 
    for t in range(1, M + 1): 
        S[t] = S[t-1] * np.exp((r - sigma ** 2 / 2) * dt + sigma * np.sqrt(dt) * rn[t]) 
    return S

In [231]:
T = 4/52
r = 0.0174
sigma = 0.3575
S0 = 3110.87
K = 3100.00

In [232]:
S = mcs_simulation_np(10000)

In [233]:
S = np.transpose(S)
S

array([[3110.87      , 3106.23151946, 3107.45977809, ..., 2399.36889172,
        2399.93221389, 2402.06405249],
       [3110.87      , 3111.80573926, 3110.97676308, ..., 3332.68265174,
        3331.86170561, 3333.65552769],
       [3110.87      , 3109.67579561, 3109.46326078, ..., 3150.02182083,
        3152.10926806, 3156.10596389],
       ...,
       [3110.87      , 3108.92975014, 3111.67376415, ..., 3389.5335016 ,
        3389.59878912, 3390.45601895],
       [3110.87      , 3113.14847642, 3109.53188787, ..., 3181.03400195,
        3180.66344082, 3178.08936392],
       [3110.87      , 3107.36887577, 3111.97859241, ..., 3444.47480119,
        3438.24871258, 3436.29735166]])

In [234]:
pp = (K - S[-1,:])>0
bpp = np.mean(np.maximum(pp.astype(int),0))
print('Binary put', str(bpp))

Binary put 0.21477852214778523


In [235]:
def delta(S, K, T, r, q, sig, payoff):
    
    #S: spot price
    #K: strike price
    #T: time to maturity
    #r: risk free rate
    #q: continuous dividend yield
    #vol: volatility of underlying asset
    #payoff: call or put
    
    d1 = (np.log(S / K) + (r - q + 0.5 * sig ** 2) * T) / (sig * np.sqrt(T))
    d2 = (np.log(S / K) + (r - q - 0.5 * sig ** 2) * T) / (sig * np.sqrt(T))
    if payoff == "call":
        delta = (np.exp(-r * T) * si.norm.pdf(d2, 0.0, 1.0)) / (sig * np.sqrt(T))
    elif payoff == "put":
        delta =  - (np.exp(-r * T) * si.norm.pdf(-d2, 0.0, 1.0)) / (sig * S * np.sqrt(T))
    
    return delta

In [236]:
delta(3110.87, 3100, 4/52, 0.0174, 0, 0.3575, 'put')


-0.0012916427665803664

In [237]:
def gamma(S, K, T, r, q, sig, payoff):
    
    d1 = (np.log(S / K) + (r - q + 0.5 * sig ** 2) * T) / (sig * np.sqrt(T))
    d2 = (np.log(S / K) + (r - q - 0.5 * sig ** 2) * T) / (sig * np.sqrt(T))
    if payoff == "call":
        gamma = - (np.exp(-r * T) * d1 * si.norm.pdf(d1, 0.0, 1.0)) / (sig**2 * S**2 * np.sqrt(T))
    elif payoff == "put":
        gamma =  (np.exp(-r * T) * d1 * si.norm.pdf(d2, 0.0, 1.0)) / (sig **2 * S**2* np.sqrt(T))
    
    return gamma

In [238]:
gamma(3110.87, 3100, 4/52, 0.0174, 0, 0.3575, 'put')

1.142564997937342e-07

In [239]:
def theta(S, K, T, r, q, sig, payoff):
    
    d1 = (np.log(S / K) + (r - q + 0.5 * sig ** 2) * T) / (sig * np.sqrt(T))
    d2 = (np.log(S / K) + (r - q - 0.5 * sig ** 2) * T) / (sig * np.sqrt(T))
    A = d1 /(2 * T)
    B = (r - q) / sig * np.sqrt(T)
    if payoff == "call":
        theta = r * np.exp(-r * T) * (d1,0.0,1.0) * si.norm.cdf(d2,0.0,1.0) + np.exp(-r * T) * si.norm.pdf(d2,0.0,1.0) * (A - B)
    elif payoff == "put":
        theta = r * np.exp(-r * T) * (1 - si.norm.cdf(-d1, 0.0, 1.0)) - (np.exp(-r * T) * si.norm.pdf(d2,0.0,1.0)) * (A - B)
    
    return theta

In [240]:
theta(3110.87, 3100.00, 4/52, 0.0174, 0, 0.3575, 'put')

-0.2400168428797643

In [241]:
def rho(S, K, T, r, q, sig, payoff):
    
    d1 = (np.log(S / K) + (r - q + 0.5 * sig ** 2) * T) / (sig * np.sqrt(T))
    d2 = (np.log(S / K) + (r - q - 0.5 * sig ** 2) * T) / (sig * np.sqrt(T))
    if payoff == "call":
        rho =  - (T) * np.exp(-r * T) * si.norm.cdf(d2, 0.0, 1.0) + (np.sqrt(T) / (sig)) * (np.exp(-r * T) * si.norm.pdf(d2,0.0,1.0))
    elif payoff == "put":
        rho = - (T) * np.exp(-r * T) * (1 - si.norm.cdf(d2, 0.0, 1.0)) - (np.sqrt(T) / (sig)) * (np.exp(-r * T) * si.norm.pdf(d2,0.0,1.0))
    
    return rho

In [242]:
rho(3110.87, 3100, 4/52, 0.0174, 0.0, 0.3575, 'put')

-0.3475209787780212

In [245]:
def vega(S, K, T, r, q, sig, payoff):
    
    d1 = (np.log(S / K) + (r -q + 0.5 * sig **2) * T) / (sig * np.sqrt(T))
    d2 = (np.log(S / K) + (r -q - 0.5 * sig **2) * T) / (sig * np.sqrt(T))
    A = np.sqrt(T)
    B = ((d2)/sig)
    if payoff == "call":
        vega = - np.exp(r * T) * si.norm.pdf(d2,0,1.0) * (A + B)
    elif payoff == "put":
        vega = np.exp(-r * T) * si.norm.pdf(d2,0,1) * (A + B)
        
    return vega    

In [253]:
vega(3110.87, 3100, 4/52, 0.0174, 0, 0.3575, 'put')

0.10963494700484258